### Soil heat and water balance in 1D 

status (20.1.2025): creates model object but does not yet simulate anything!

### pyAPES.soil

- Class *pyAPES.soil.Soil_1D* computes coupled soil water and heat flows in unevenly-spaced vertical domain with variable upper and lower boundary conditions. It uses two sub-models *Water_1D* and *Heat_1D*

- *pyAPES.soil.water.Water_1D* contains two alternative functions to solve soil water budget: 
    1. *waterFlow1D* computes water flux between layers explicitly calculated using Richards equation. The solution is based on the implicit finite differnce scheme of van Dam & Feddes (1998) J. Hydrol., 
    2. *waterStorage1D* tracks soil profile water storage and assumes water potential is at hydraulic equilibrium above the (shallow) water table level (WTL). This means soil volumetric water content (SWC) at each layer is uniquely determined by local hydraulic properties (pF-curve) and distance to WTL.

    Soil hydraulic properties are described using van Genuchten et al. (xxxx). Ditch drainage is computed via Hooghoud-equation.

- *pyAPES.soil.heat.Heat_1D* solves soil heat conduction and temperature profile usign Fourier equation with phase-changes. Soil freezing/thawing and resulting liquid/ice phase water contents are described using empirical freezing curve.

In [3]:
# setting path
import sys
#sys.path.append('c:\\Repositories\\pyAPES_main')
import os
from dotenv import load_dotenv

load_dotenv()
pyAPES_main_folder = os.getenv('pyAPES_main_folder')

sys.path.append(pyAPES_main_folder)
#print(sys.path)

# imports
import numpy as np
import matplotlib.pyplot as plt

In [6]:
# import soilprofile submodel
from pyAPES.soil.soil import Soil_1D

# import water submodel and Richards equation solver - can be used independently
from pyAPES.soil.water import Water_1D, waterFlow1D

# import heat submodel and Fourier equation solver
from pyAPES.soil.heat import Heat_1D, heatflow1D

In [10]:
# read forcing file from US-Prr & create forcing for soil heat and water flow
from pyAPES.utils.iotools import read_forcing

ffile = r'forcing/US-Prr/US-Prr_forcing_2011_2015.dat'

forcing = read_forcing(os.path.join(pyAPES_main_folder, ffile), sep=';', start_time='2011-01-01', end_time='2012-01-01')

In [11]:
forcing.head()

,doy,Prec,P,Tair,Tdaily,U,Ustar,H2O,CO2,Zen,LWin,diffPar,dirPar,diffNir,dirNir,X,DDsum
2011-01-01 00:00:00,1,0.0,97900.0,-15.445,-11.234,1.19,0.136,0.001607,390.0,2.383,243.020,0.0,0.0,0.0,0.0,-25.395,0.0
2011-01-01 00:30:00,1,0.0,98000.0,-15.643,-11.234,0.27,0.055,0.001574,390.0,2.400,239.907,0.0,0.0,0.0,0.0,-25.395,0.0
2011-01-01 01:00:00,1,0.0,98000.0,-15.310,-11.234,0.27,0.031,0.001606,390.0,2.407,240.238,0.0,0.0,0.0,0.0,-25.395,0.0
2011-01-01 01:30:00,1,0.0,98000.0,-15.683,-11.234,0.27,0.031,0.001568,390.0,2.405,247.739,0.0,0.0,0.0,0.0,-25.395,0.0
2011-01-01 02:00:00,1,0.0,98000.0,-15.433,-11.234,0.27,0.031,0.001597,390.0,2.393,243.189,0.0,0.0,0.0,0.0,-25.395,0.0


### Parameterize soil sub-model for US-Prr

In [15]:
# --- Soil water & heat: pyAPES.soil.Soil

# grid and soil properties: pF and conductivity values from Launiainen et al. (2015), Hyytiala

soil_grid = {#thickness of computational layers [m]: 0.01 m until 0.1m, 0.02m until 0.5m, 0.05 m until 2m and thereafter 0.2m
            'dz': [0.01] * 10 + [0.02] * 20 + [0.05] * 30 + [0.2]*20,
            # bottom depth of layers with different characteristics [m]
            'zh': [-0.05, -0.1, -0.35, -10.0]
            }

soil_properties = {
    # Liu & Lennartz 2019 Sphagnum type II, III, V & Launiainen et al. 2022 cluster C1: silty soil with high org. content
    'pF': {  # vanGenuchten water retention parameters
           'ThetaS': [0.80, 0.8, 0.75, 0.6], # [m3m-3]
           'ThetaR': [0.001, 0.001, 0.001, 0.001], # [m3m-3]
           'alpha': [0.16, 0.22, 0.02, 1.12], # [cm-1]
           'n': [1.25, 1.21, 1.28, 4.45] # [-]
    },
    'saturated_conductivity_vertical': [1e-4, 1e-5, 1e-6, 1e6],  # saturated vertical hydraulic conductivity [m s-1]
    'saturated_conductivity_horizontal': [1e-4, 1e-5, 1e-6, 1e6],  # saturated horizontal hydraulic conductivity [m s-1]
    'solid_heat_capacity': None,  # [J m-3 (solid) K-1] - if None, estimated from organic/mineral composition
    'solid_composition': { # fraction of solids
        'organic': [1.0, 1.0, 1.0, 0.2],
        'sand': [0.0, 0.0, 0.0, 0.0],
        'silt': [0.0, 0.0, 0.0, 0.4],
        'clay': [0.0, 0.0, 0.0, 0.2],
    },
    'freezing_curve': [0.2, 0.2, 0.2, 0.4],  # freezing curve parameter, for peat see Nagare et al. 2012 HESS https://doi.org/10.5194/hess-16-501-2012
    'bedrock': {
        'solid_heat_capacity': 2.16e6,  # [J m-3 (solid) K-1]
        'thermal_conductivity': 3.0  # thermal conductivity of non-porous bedrock [W m-1 K-1]
    }
}

# --- water model: pyAPES.soil.water.Water

water_model = {'solve': True,
               'type': 'Equilibrium',  # solution approach 'Richards' | 'Equilibrium'
               'pond_storage_max': 0.05,  #  maximum pond depth [m]
               'initial_condition': {
                       'ground_water_level': -0.05,  # groundwater depth [m], <=0
                       'pond_storage': 0.0  # pond depth at surface [m]
                       },
               'lower_boundary': {
#                       'type': 'head_oneway',
#                       'value': -0.0,
                       'type': 'impermeable',
                       'value': None,
                       'depth': -5.0
                       },
               'drainage_equation': {
                       'type': None,  #
#                       'type': 'Hooghoudt',  #
#                       'depth': 1.0,  # drain depth [m]
#                       'spacing': 45.0,  # drain spacing [m]
#                       'width': 1.0,  # drain width [m]
                       }
                }

# --- heat model: pyAPES.soil.heat.Heat
heat_model = {'solve': True,
              'initial_condition': {
                      'temperature': -2.0,  # initial soil temperature [degC], assumed constant with dept - can also be array of correct length.
                      },
              'lower_boundary': {  # lower boundary condition (type, value)
                      'type': 'temperature',
                      'value': -2.0
                      },
              }

# --- compile soil model parameter dictionary
spara = {'grid': soil_grid,
         'soil_properties': soil_properties,
         'water_model': water_model,
         'heat_model': heat_model}

In [16]:
# create Soil1D model

model = Soil_1D(spara)

In [14]:
model.__dict__

{'solve_heat': True,
 'solve_water': True,
 'Nlayers': 70,
 'grid': {'z': array([-0.005, -0.015, -0.025, -0.035, -0.045, -0.055, -0.065, -0.075,
         -0.085, -0.095, -0.11 , -0.13 , -0.15 , -0.17 , -0.19 , -0.21 ,
         -0.23 , -0.25 , -0.27 , -0.29 , -0.31 , -0.33 , -0.35 , -0.37 ,
         -0.39 , -0.41 , -0.43 , -0.45 , -0.47 , -0.49 , -0.525, -0.575,
         -0.625, -0.675, -0.725, -0.775, -0.825, -0.875, -0.925, -0.975,
         -1.025, -1.075, -1.125, -1.175, -1.225, -1.275, -1.325, -1.375,
         -1.425, -1.475, -1.525, -1.575, -1.625, -1.675, -1.725, -1.775,
         -1.825, -1.875, -1.925, -1.975, -2.1  , -2.3  , -2.5  , -2.7  ,
         -2.9  , -3.1  , -3.3  , -3.5  , -3.7  , -3.9  ]),
  'dz': array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.02,
         0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02,
         0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.05, 0.05, 0.05,
         0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.0